# Exploring the ACLED API

The dataMisc function contains code to search the ACLED API

In [3]:
import sys, os, requests, json
import pycountry

import pandas as pd
import geopandas as gpd

sys.path.insert(0, "../src")

import GOSTrocks.dataMisc as dMisc

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
creds = dMisc.get_acled_creds()
email = creds['email']
api_key = creds['api_key']

iso3 = "KEN"
country_coding = pycountry.countries.get(alpha_3=iso3)
iso_code = country_coding.numeric
start_date = "2024-01-01"


In [5]:
# Fetch all ACLED events since 2024-01-01 in selected country
acled_data = dMisc.acled_search(api_key, email, iso3=iso_code, start_date=start_date)
acled_data.groupby(['year', 'event_type']).count()['event_date']

year  event_type                
2024  Battles                        178
      Explosions/Remote violence      25
      Protests                      1671
      Riots                          917
      Strategic developments         291
      Violence against civilians     273
2025  Battles                         18
      Explosions/Remote violence       4
      Protests                       257
      Riots                          159
      Strategic developments          42
      Violence against civilians      62
Name: event_date, dtype: int64

In [6]:
# Fetch adm1 boundaries for selected ISO3
# adm0 = dMisc.get_geoboundaries(iso3, "ADM0")
adm1 = dMisc.get_geoboundaries(iso3, "ADM1")

In [7]:
# Query all events that are within the capital
query_geom = adm1.loc[adm1['shapeName'] == "Nairobi"].union_all()
acled_data = dMisc.acled_search(api_key, email, bounding_box=query_geom.bounds)
acled_data.groupby(['year', 'event_type']).count()['event_date']

C:\Users\WB411133\AppData\Local\Temp\ipykernel_32752\1060312689.py:2: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  query_geom = adm1.loc[adm1['shapeName'] == "Nairobi"].unary_union


year  event_type                
1997  Battles                        4
      Explosions/Remote violence     1
      Protests                       8
      Riots                         17
      Strategic developments         1
                                    ..
2025  Battles                        1
      Protests                      47
      Riots                         31
      Strategic developments        13
      Violence against civilians     8
Name: event_date, Length: 147, dtype: int64

In [8]:
# Query all events that are within the capital with explicit field selection
query_geom = adm1.loc[adm1['shapeName'] == "Nairobi"].union_all()
### Defining the fields to be returned shortens the response time from 10.6 seconds to 6.2 seconds
cled_data = dMisc.acled_search(api_key, email, bounding_box=query_geom.bounds, fields=['year','event_type', 'geo_precision'])
acled_data.groupby(['year', 'event_type']).count()

geo_precision
year event_type                               
1997 Battles                                 4
     Explosions/Remote violence              1
     Protests                                8
     Riots                                  17
     Strategic developments                  1
...                                        ...
2025 Battles                                 1
     Protests                               47
     Riots                                  31
     Strategic developments                 13
     Violence against civilians              8

[147 rows x 1 columns]

In [ ]:
acled_data